# Simplest Example of TCOD

In [1]:
import tcod
tcod.console_set_custom_font(b"gui/data/fonts/terminal8x8_gs_tc.png", tcod.FONT_LAYOUT_TCOD | tcod.FONT_TYPE_GREYSCALE)


In [ ]:
tcod.console_init_root(80, 50, b"Welcome to the Island", False, tcod.RENDERER_SDL)

In [1]:
%pwd

'/home/millejoh/Dropbox/Projects/Islands'

In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np

# Scaffolding

Either call the below or execute the code in Island_Manual.org

In [11]:
import ctypes
import tcod
from tcod.console import Console
#from tcod.console import ConsoleCell
from gui.window import Window, Viewport, ListWindow
from gui.managers import WindowManager, GuiEventLoop
import matplotlib.pyplot as plt
#from MapChunk import MapChunk, gEntity
#from pyDatalog import pyDatalog

In [12]:
c_init = [[' ']*20]*10
c = np.empty_like(c_init)
c

array([['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '']], dtype='<U1')

In [5]:
np.ravel?

In [13]:
a = np.zeros((10,10,3))
a[0,0] = tcod.white

In [14]:
a[..., 2].ravel()

array([255.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.])

## LibTCOD

Testing RootConsole, basic gui event loop and windows. When running this
double check that:

 1. Windows are draggable.
 2. Windows can be resized.
 3. Window focus changes on click.
 4. Window close button works.


The New Manager API
-------------------

Because I am insane, and felt that far too many global variables were
being used to store and communicate state I have factored out event
loop and window management function into two Python classes:
- `WindowManager`
- `GuiEventLoop`



In [1]:
from worldgen import WorldGenerator

In [2]:
w = WorldGenerator(50, 80)

In [3]:
%%timeit

w = WorldGenerator(100, 100)


24.7 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit

w.build_base_map(600)


In [ ]:
%%timeit

w.compute_precipitation()


In [6]:
%%timeit

w.erode_map()


849 ms ± 51.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

w.smooth_map()

310 µs ± 2.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%run demo_test.py
#from worldgen import WorldGenerator

In [ ]:
%debug

In [3]:
tcod.loader.lib.TCOD_console_set_dirty

<Lib object for 'tcod._libtcod'>

### Regression Testing

In [14]:
from gui.window import Window, Viewport, ListWindow
from gui.managers import WindowManager
from gui.console import ConsoleCell

In [2]:
wm = WindowManager(80, 60)

In [11]:
w = Window(5, 5, 15, 15,title='Window 1', framed=True, window_manager=wm)
w2 = Window(10, 10, 20, 30, title='Window 2', framed=True, window_manager=wm)

In [4]:
w2.unhide(False)

In [5]:
v = Viewport(tlx=10, tly=10, width=40, height=40, framed=True, map_width=120,
             map_height=120, view_tlx=0, view_tly=0, title='The View',
             window_manager=wm)

In [6]:
v.map_console[1,1]

ConsoleCell(symbol=' ', foreground=Color(255, 255, 255), background=Color(0, 0, 0))

In [7]:
v.map_console[1,1] = ConsoleCell('.', 'white', 'black')

In [8]:
v[1,1] = '!'

In [12]:
wm.all_windows()


[Window(10,10,20,30,title='Window 2'),
 Window(5,5,15,15,title='Window 1'),
 Viewport(10,10,40,40,title='The View'),
 Window(10,10,20,30,title='Window 2'),
 Window(5,5,15,15,title='Window 1')]

In [16]:
list_view = ListWindow(tlx=15, tly=15, width=20, height=5, title='List Window', framed=True, window_manager=wm)
list_view.add_item('An item.', 'An item.')

In [17]:
list_view.add_item('Another item.', 'Uhuh.')
list_view.add_item('Keep going.', 'Number.')
list_view.add_item('Scrolling yet?', 'Scroll')
list_view.add_item('last one', 'the end.')

In [18]:
list_view.width - (0 if list_view.use_borders else 2)

18

In [19]:
list_view.wrap_items = True

# TCOD Utilities in Python

# Representing the World

Need to represent:
 1. Terrain.
 2. Objects (walls, doors, tables, trees, etc.)
 3. Actors

In terms of drawing, terrain type (or biome) determines background color of a cell.

From libtcod's wordlgen exampe here is a example of biome/terrain types arranged
in a matrix (Whittaker diagram). Columns are precipitation levels, rows are temperature.



[Polygonal map generation]
(http://www-cs-students.stanford.edu/~amitp/game-programming/polygon-map-generation/)
is a neat page and discusses biomes. It bins elevations and moisture levels to
provide to following matrix:



# Unit Tests for the TCOD GUI

Really need some good unit tests for window management, i.e. managing the window
stack (`WindowStack`) and determining when windows are touching.

### Events

### Colors

In [11]:
import gui.color

gui.color.decompose_color(0x010203)

(3, 2, 1)

In [12]:
gui.color.compose_color(3, 2, 1) == 0x010203

True

### Colored Strings

In [3]:
import gui.string
import tcod

t1 = "Lisp is the {red}red{/} pill."
t2 = "G{'i}mli, son of Glo{'i}n."
t3 = "{fg:blue}Python{/} is also quite {bg:green}nice{/}."
t4 = "A test setting by foreground and background {fg:cyan,bg:grey}color{/}."
t5 = "{click:LABEL}Click me.{/}"

In [4]:
for val in gui.string.next_token(t4, gui.string.master_pat):
    print(val)

Token(type='TEXT', value='A test setting by foreground and background ')
Token(type='COLOR', value='{fg:cyan,bg:grey}')
Token(type='TEXT', value='color')
Token(type='MU_TERM', value='{/}')
Token(type='TEXT', value='.')


In [5]:
mu1 = '{fg:blue}Python{/}'

In [6]:
gui.string.color_to_control_string(255)

'\x06ÿ\x01\x01'

In [7]:
'{:c}'.format(tcod.COLCTRL_FORE_RGB)

'\x06'

In [8]:
[tok for tok in gui.string.next_token('fg:blue', gui.string.color_pat)]

[Token(type='FG_COLOR', value='fg:'), Token(type='COLOR', value='blue')]

In [9]:
gui.string.make_colored_string(t3)

'\x06\x01\x01ÿPython\x08 is also quite \x07\x01\x80\x01nice\x08.'

### Testing Low-level Interface

In [0]:
tcod.console_init_root(80, 60, b"Root")

In [2]:
tcod

NameError: name 'tcod' is not defined

In [22]:
wg = WorldGenerator(40, 40)

In [35]:
wg._hm

array(Heightmap(40, 40), dtype=object)

In [30]:
wg.build_base_map(30)

In [31]:
wg.compute_precipitation()

In [32]:
wg.erode_map()

In [33]:
wg.smooth_map()

AttributeError: 'numpy.ndarray' object has no attribute 'kernel_transform'

In [34]:
wg._hm.kernel_transform()

AttributeError: 'numpy.ndarray' object has no attribute 'kernel_transform'

In [ ]:
tcod.mouse_